In [1]:
import os
import sys
import mlflow
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from logger.logger import logging
from exception.exception import customexception
from dataclasses import dataclass
from pathlib import Path
from utils.common import create_directories,read_yaml,load_object,read_csv,save_json

In [2]:
#config_entity -> configuration_manager -> component_file -> pipeline -> main

In [3]:
# config_entity
@dataclass(frozen=True)
class ModelEvaluationConfig:
    trained_model_path: Path
    test_data_path: Path
    metrics_json_path: Path

In [4]:
# configuration_menager
from constants import CONFIG_FILE_PATH
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self)->ModelEvaluationConfig:
        training_config = self.config.model_training
        datatransform_config = self.config.data_transformation
        model_evaluation_config = self.config.model_evaluation
        
        create_directories([model_evaluation_config.root_dir])
        evaluation_config = ModelEvaluationConfig(
            trained_model_path=training_config.model_path,
            test_data_path=datatransform_config.transform_data_files,
            metrics_json_path=model_evaluation_config.metrics_path
        )
        return evaluation_config
        

In [5]:
# component file
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self,y_pred,y_test):
        mse = mean_squared_error(y_test,y_pred)
        logging.info(f"Mean squared error of model is: {mse}")
        mae = mean_absolute_error(y_test,y_pred)
        logging.info(f"Mean absolute error of model is: {mae}")
        score_r2 = r2_score(y_test,y_pred)
        logging.info(f"r2 score of model is: {score_r2}")
        
        logging.info("Storing metrics into json file")
        metrics = {"MSE":mse,"MAE":mae,"r2 score":score_r2}
        json_filepath = Path(self.config.metrics_json_path)
        # print(json_filepath)
        save_json(json_filepath,metrics)
        return mse,mae,score_r2
    
    def model_evaluate(self):
        logging.info("Loading the model")
        model_path = self.config.trained_model_path
        model_obj = load_object(model_path)
        
        logging.info("Loading the testing data")
        test_data_path = os.path.join(self.config.test_data_path,"transform_test.csv")
        test_df = read_csv(test_data_path)
        
        X_test = test_df.drop('price',axis=1)
        y_test = test_df['price']
        
        logging.info("Evaluating data")
        y_pred = model_obj.predict(X_test)
        
        mse,mae,score_r2 = self.eval_metrics(y_pred,y_test)

In [9]:
# cd ..

In [11]:
try:
    logging.info("Model evaluation started")
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.model_evaluate()
    logging.info("Model evaluation completed")
except Exception as e:
    raise customexception(e,sys)

artifacts\model_evaluation\metrics.json
